# Milestone 2

## Introduction

We are going to analyze the dataset, cleanliness, blabla, start to explore the question.

ntm lucas

## Dataset overview

Size of dataset, blablabla

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import re

In [ ]:
OPEN_FOOD_FACTS_PATH = 'data/en.openfoodfacts.org.products.csv'
food_facts_df = pd.read_csv(OPEN_FOOD_FACTS_PATH, delimiter='\t')
food_facts_df.head(5)

### Available fields

The dataset description is available [here](https://static.openfoodfacts.org/data/data-fields.txt).



This dataset is provided with a text file describing the different fields as follows :

* fields that end with _t are dates in the UNIX timestamp format (number of seconds since Jan 1st 1970)
* fields that end with _datetime are dates in the iso8601 format: yyyy-mm-ddThh:mn:ssZ
* fields that end with _tags are comma separated list of tags (e.g. categories_tags is the set of normalized tags computer from the categories field)
* fields that end with a language 2 letter code (e.g. fr for French) is the set of tags in that language
* fields that end with _100g correspond to the amount of a nutriment (in g, or kJ for energy) for 100 g or 100 ml of product
* fields that end with _serving correspond to the amount of a nutriment (in g, or kJ for energy) for 1 serving of the product

**TODO**: show data

### Data completeness

Sparse dataset, lots of quasi empty columns

### Reliability of the dataset

crowd-sourced, blabla

## Question 1: How do nutrients influence the selected nutrition scores?

### Additional cleaning

### Data exploration

### Results

quick conclusion, we keep the question, further investigations

## Question 2: Comparing bio vs. normal products, is there a real difference in composition?

### Additional cleaning

### Data exploration

### Results

quick conclusion, we keep the question, further investigations

## Question 3: What are the levels of interdependencies between nations based on their production and importations?

### Additional cleaning

### Data exploration

### Results

quick conclusion, we **don't keep** the question, further investigations

## Conclusion: proposed data story